In [1]:
import netgen.gui
from ngsolve import *
from ngsolve.internal import visoptions
from netgen.meshing import ImportMesh

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2304
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 10 thread(s)


2024-09-26 22:31:53.243 Python[48171:1387359] CGSSetSurfaceColorSpace failed - error 1000 (windowID:15240)
2024-09-26 22:31:53.243 Python[48171:1387359] CGSSetSurfaceColorSpace failed - error 1000 (windowID:15240)
2024-09-26 22:31:53.354 Python[48171:1387359] +[IMKClient subclass]: chose IMKClient_Legacy
2024-09-26 22:31:53.355 Python[48171:1387359] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [2]:
ngmesh = ImportMesh('cell_mesh_R1_opt.mesh')

In [3]:
from netgen.meshing import FaceDescriptor
isinplane = lambda el: all((((ngmesh[v][2] < -27)) for v in el.vertices))
fd_id = ngmesh.Add(FaceDescriptor(surfnr=2,domin=1,domout=0,bc=2))
for el in ngmesh.Elements2D():
    if isinplane(el):
        el.index = fd_id
    else:
        el.index = fd_id-1
ngmesh.SetBCName(fd_id-1, "bottom")
ngmesh.SetBCName(fd_id-2, "surface")

In [4]:
mesh = Mesh(ngmesh)
Draw(mesh)

In [298]:
#---------------- parameters ------------------
import numpy as np
Pi = np.pi
orderset = 2
dimset = mesh.dim
v = 0.5
Yg = (sqrt(x**2+y**2+z**2))**(-1)
Pout = 30
pz = -27
cz = 16
thick = 0.5
G =  Pout/Integrate(thick,mesh,BND)
Yf = 300
rho1 = Yf * cos(x**2+y**2)
rho2 = Yf * cos(x**2+y**2)
rho3 = Yf * cos(x**2+y**2)

In [299]:
fes = H1(mesh, order=orderset,  dirichlet="bottom", dim=dimset)
u = fes.TrialFunction()
nsurf = - specialcf.normal(dimset)

In [300]:
u_grad = grad(u).Trace()
Eu = 1/2*(u_grad.trans * u_grad - Id(dimset))
Cg = CoefficientFunction(((Yg,v*Yg,0),(Yg,v*Yg,0),(0,0,Yg*(v-1)/2)),dims=(dimset,dimset))
Cf = CoefficientFunction(((rho1,rho2,0),(rho2,rho1,0),(0,0,rho3)),dims=(dimset,dimset))
C = Cg 
ngradu = u_grad.trans * nsurf
beta = Pout*nsurf
Phi = Eu.trans * (C * Eu)
a = BilinearForm(fes, symmetric=False)
a += Variation(Trace(Phi).Compile()*ds)
a += Variation((ngradu-beta)*(ngradu-beta)*ds)

In [301]:
u = GridFunction(fes)
du = GridFunction(fes)
res = u.vec.CreateVector()
potensial = G*(z-pz)*(z-cz)*nsurf 
ini = CoefficientFunction([potensial if bc == 'surface' else (0,0,0) for bc in mesh.GetBoundaries()])
u.Set(ini,definedon=fes.mesh.Boundaries("surface|bottom"))
Draw(u, mesh, 'disp')

In [302]:
step = 1e-2
for i in range(10):
   print ("Damped Newton iteration", i+1)
   print ("  energy = ", a.Energy(u.vec))
   a.Apply(u.vec, res) 
   a.AssembleLinearization(u.vec)
   du.vec.data = a.mat.Inverse(fes.FreeDofs()) * res
   r =  InnerProduct(du.vec, res)
   print ("  err^2 =", abs(r))
   u.vec.data -= step * du.vec.data
   Draw(u, mesh, 'disp')
   if abs(r) < 1e-6:
       break

Damped Newton iteration 1
  energy =  22679839.99445434
  err^2 = 0.5711997917756256
Damped Newton iteration 2
  energy =  22679840.00013771
  err^2 = 0.5596260409858544
Damped Newton iteration 3
  energy =  22679840.00570609
  err^2 = 0.5482909062521378
Damped Newton iteration 4
  energy =  22679840.011161484
  err^2 = 0.5371893006258609
Damped Newton iteration 5
  energy =  22679840.016506705
  err^2 = 0.526316252595054
Damped Newton iteration 6
  energy =  22679840.021743514
  err^2 = 0.5156669031631562
Damped Newton iteration 7
  energy =  22679840.02687435
  err^2 = 0.5052365030152323
Damped Newton iteration 8
  energy =  22679840.031901434
  err^2 = 0.49502040976839534
Damped Newton iteration 9
  energy =  22679840.03682692
  err^2 = 0.4850140853033471
Damped Newton iteration 10
  energy =  22679840.041652802
  err^2 = 0.47521309317405225
